# Deploy SageMaker trained AMPLIFY model on Sagemaker Inference

### Important: Setting the Jupyter Working Directory

This notebook assumes that your working directory is set to the following path: `<repo-root>/framework-integrations/sagemaker/training`

#### How to Check and Set the Working Directory
Before running the notebook, you can verify that your working directory is correct by running the following command:

In [2]:
import os
print(os.getcwd())

/home/ec2-user/SageMaker/AMPLIFY-AWS/framework-integrations/sagemaker/inference


If the output is not `<repo-root>/framework-integrations/sagemaker/inference`, you can set the working directory manually by running:

In [ ]:
os.chdir('/path/to/your/repository/framework-integrations/sagemaker/inference')

### Setup Environment

First, ensure that the necessary packages are installed:

In [ ]:
!pip install boto3 sagemaker -U -q

### Initialize your SageMaker session and IAM role

Before deploying the model, you need to set up the SageMaker session and specify your IAM role

In [12]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
#from sagemaker.huggingface import HuggingFaceModel
from sagemaker.pytorch import PyTorchModel

sagemaker_session = Session()
region = boto3.Session().region_name
execution_role = get_execution_role()
region = boto3.Session().region_name

### Specify the model ID and the instance details

In [20]:
instance_type = 'ml.g5.4xlarge'

### Specify the container image for the inference

In [21]:
image = image_uris.retrieve(region=region, framework='pytorch', image_scope='inference', version='2.3', base_framework_version='pytorch2.0.0', instance_type=instance_type)
image

'763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:2.3-gpu-py311'

### Specify the trained model data location

In [43]:
#s3_model_data = "s3://sagemaker-us-west-2-111918798052/dummy-model.tar.gz"  ## Change this accordingly
s3_model_data = "s3://sagemaker-us-west-2-111918798052/AMPLIFY-hf-training-job-1729483946/output/model.tar.gz"  ## Change this accordingly

### Deploy the Model with the Custom Inference Script

In [44]:
sm_model = PyTorchModel(
    model_data=s3_model_data,
    role=execution_role,
    image_uri=image,
    source_dir="code/sm-trained/",
    entry_point="inference_sm_trained.py",
)

In [45]:
endpoint_name = f"AMPLIFY-sm-trained-endpoint"

predictor = sm_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name
)

--------------!

### Make a single prediction 

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()  
predictor.deserializer = JSONDeserializer()  

sequence = "MSVVGIDLGFQSCYVAVARAGGIETIANEYSDRCTPACISFGPKNR"

result = predictor.predict({"sequence": sequence})

print(f"Logits: {result}")

### Delete Endpoint

In [47]:
predictor.delete_endpoint()